In [6]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [7]:
# Order

o_columnnames = ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"]

for i in range(len(o_columnnames)):
    o_columnnames[i] = o_columnnames[i].lower()
    
o_data_types = {
    'o_orderkey': int,
    'o_custkey': int,
    'o_orderstatus': str,
    'o_totalprice': float,
    'o_orderpriority': str,
    'o_clerk': str,
    'o_shippriority': int,
    'o_comment': str
}

o_parse_dates = ['o_orderdate']

# Don't set indexes, as we can't access them with Pandas selection!
orders = pd.read_table("../data/orders.tbl.csv", sep="|", names=o_columnnames, dtype=o_data_types, parse_dates=o_parse_dates)

In [8]:
start_time = time.time()

df_filter_1 = orders[['o_orderkey', 'o_custkey', 'o_orderstatus', 'o_totalprice', 'o_orderdate', 'o_orderpriority', 'o_clerk', 'o_shippriority', 'o_comment']]
df_filter_1['before_1'] = (df_filter_1.o_custkey)
df_filter_1['before_2'] = (df_filter_1.o_totalprice)
df_filter_1['before_3'] = (df_filter_1.o_orderkey)
df_filter_1['before_4'] = (df_filter_1.o_shippriority)
df_filter_1['before_1'] = (df_filter_1.o_custkey)
df_filter_1['before_2'] = (df_filter_1.o_totalprice)
df_filter_1['before_3'] = (df_filter_1.o_orderkey)
df_filter_1['before_4'] = (df_filter_1.o_shippriority)
df_group_1 = df_filter_1 \
    .groupby(['o_custkey']) \
    .agg(
        count_before_1=("before_1", "count"),
        mean_before_2=("before_2", "mean"),
        sum_before_3=("before_3", "sum"),
        min_before_4=("before_4", "min"),
        max_before_1=("before_1", "max"),
        min_before_2=("before_2", "min"),
        max_before_3=("before_3", "max"),
        mean_before_4=("before_4", "mean"),
    )
df_group_1['customer'] = (df_filter_1.o_custkey)
df_group_1['fun_aggregate'] = (((df_group_1.count_before_1 + df_group_1.mean_before_2) / (df_group_1.sum_before_3 + df_group_1.min_before_4)) * 25)
df_group_1['massive_query'] = (((df_group_1.max_before_1 * df_group_1.min_before_2) * (df_group_1.max_before_3 - df_group_1.mean_before_4)) + 5)
df_group_1 = df_group_1[['customer', 'fun_aggregate', 'massive_query']]

end_time = time.time()
print(df_group_1)

print("--- %s seconds ---" % (end_time - start_time))

           customer        fun_aggregate          massive_query
o_custkey                                                      
1             78002  0.12810308393643452        277457229149.34
2            123314   0.2728328523217036     131517134132.36002
4             44485 0.060140050380629084          25393471717.0
5             55624   1.0575971082868225         179142719014.1
7            130057  0.09635563593962534     1129452816091.3499
...             ...                  ...                    ...
149993        34129   0.2114397542112407     6986911012645099.0
149995        94618 0.054745827654903836     1835363180785452.0
149996        87190  0.14529664144308146  5.241800038924327e+16
149998        62458  0.04995259185356739 1.7912452741612304e+16
149999        24202 0.055579803733751744     9985699992814908.0

[99996 rows x 3 columns]
--- 0.46320533752441406 seconds ---


In [9]:
# Pandas Code for:
    # SELECT (COUNT ( o_custkey * (o_totalprice / -1) ) + AVG ( o_totalprice )) FROM orders GROUP BY o_custkey;

start_time = time.time()
df_filter_1 = orders[['o_orderkey', 'o_custkey', 'o_orderstatus', 'o_totalprice', 'o_orderdate', 'o_orderpriority', 'o_clerk', 'o_shippriority', 'o_comment']]
df_filter_1['before_1'] = ((df_filter_1.o_custkey) * ((df_filter_1.o_totalprice) / 2))
df_filter_1['before_2'] = (df_filter_1.o_totalprice)
df_group_1 = df_filter_1 \
    .groupby(['o_custkey']) \
    .agg(
        count_before_1=("before_1", "count"),
        mean_before_2=("before_2", "mean"),
    )
df_group_1['counto_custkeyo_totalprice2avgo_totalprice'] = (df_group_1.count_before_1 + df_group_1.mean_before_2)
df_group_1 = df_group_1[['counto_custkeyo_totalprice2avgo_totalprice']]

end_time = time.time()
print(df_group_1)

print("--- %s seconds ---" % (end_time - start_time))


           counto_custkeyo_totalprice2avgo_totalprice
o_custkey                                            
1                                           97966.485
2                                  146903.20428571428
4                                         132446.8395
5                                           171245.32
7                                         184882.3225
...                                               ...
149993                             169943.62833333333
149995                             119301.31052631578
149996                                      160294.37
149998                             145278.90120000002
149999                             171159.29727272727

[99996 rows x 1 columns]
--- 0.3038599491119385 seconds ---


In [10]:
df_filter_1 = orders[['o_orderkey', 'o_custkey', 'o_orderstatus', 'o_totalprice', 'o_orderdate', 'o_orderpriority', 'o_clerk', 'o_shippriority', 'o_comment']]
df_filter_1['before_1'] = (df_filter_1.o_orderkey)
df_group_1 = df_filter_1 \
    .groupby(['o_custkey']) \
    .agg(
        count_before_1=("before_1", "count"),
    )
df_group_1['count_orders'] = df_group_1.count_before_1
df_group_1 = df_group_1[['count_orders']]

print(df_group_1)

           count_orders
o_custkey              
1                     6
2                     7
4                    20
5                     4
7                    16
...                 ...
149993                6
149995               19
149996                8
149998               25
149999               22

[99996 rows x 1 columns]
